#### 가장 많은 빈도 수의 2268 X 4032 이미지만 선택하고 나머지는 버림
#### 원본 비율을 유지한 채 사이즈 변경 (360 X 640)  

이론상, 다른 사이즈의 이미지도 사용할 수는 있을 듯 하지만 테스트는 안해본 상태.  
일단 모두 높이를 640으로 통일해야 모델 입력으로 쓸 수 있을 듯 하다.  

전처리 순서  
1. selective_resizing
2. format_translation : trans_format_without_padding 함수 사용
3. split_dataset

In [1]:
import os
import shutil
import numpy as np
from PIL import Image, ExifTags


root_path = "D:/Downloads/street-facilities-raw"

folders = ["09.braille-block", "12.sidewalk-block", "13.bicycle-road"]
sub_folders= ["bad", "good"]

# folders = ["09.braille-block"]
# sub_folders= ["bad"]

save_folder = "D:\Downloads\street-facilities-selected"


for folder in folders:
    for sub_folder in sub_folders:
        image_path = f"{root_path}/images/{folder}/{sub_folder}"
        print(image_path) # 진행상황

        image_names = [f for f in os.listdir(image_path) if f.endswith('.jpg') or f.endswith('.jpeg')] # .jpeg 도 극히 일부 존재

        cnt = len(image_names)
        idx = 0
        for image_name in image_names:
            idx += 1
            if idx % 100 == 1:
                print(f"{idx} / {cnt}") # 진행 상황

            image = Image.open(os.path.join(image_path, image_name))
            width, height = image.size

            json_file = os.path.splitext(image_name)[0] + ".json" # 이미지 확장자명(.jpg or .jpeg) 제거
            
            # 조건에 해당하는 이미지 사이즈 변경 후 저장
            if ((height == 2268 and width == 4032) or (height == 4032 and width == 2268)): 
                exif_data = image.getexif()

                # orientation = None
                # for tag, value in exif_data.items():
                #     if ExifTags.TAGS.get(tag) == "Orientation":
                #         orientation = value
                #         break

                orientation = exif_data.get(274)  # Orientation 태그는 274번

                # 방향 정보에 따라 이미지 회전
                if orientation == 3:
                    image = image.rotate(180, expand=True)
                elif orientation == 6:
                    image = image.rotate(270, expand=True)
                elif orientation == 8:
                    image = image.rotate(90, expand=True)

                # 사이즈 변경
                new_width = int(np.ceil(640 * 2268/4032))
                new_height = 640
                resized_image = image.resize((new_width, new_height), Image.LANCZOS)

                save_path = f"{save_folder}/images/{image_name}"
                resized_image.save(save_path)
                # 라벨 (아직 json) 도 함께 copy 해서 저장
                shutil.copy(f"{root_path}/labels/{folder}/{sub_folder}/{json_file}", os.path.join(save_folder, "labels"))

D:/Downloads/street-facilities-raw/images/09.braille-block/bad
1 / 1578
101 / 1578
201 / 1578
301 / 1578
401 / 1578
501 / 1578
601 / 1578
701 / 1578
801 / 1578
901 / 1578
1001 / 1578
1101 / 1578
1201 / 1578
1301 / 1578
1401 / 1578
1501 / 1578
D:/Downloads/street-facilities-raw/images/09.braille-block/good
1 / 504
101 / 504
201 / 504
301 / 504
401 / 504
501 / 504
D:/Downloads/street-facilities-raw/images/12.sidewalk-block/bad
1 / 3624
101 / 3624
201 / 3624
301 / 3624
401 / 3624
501 / 3624
601 / 3624
701 / 3624
801 / 3624
901 / 3624
1001 / 3624
1101 / 3624
1201 / 3624
1301 / 3624
1401 / 3624
1501 / 3624
1601 / 3624
1701 / 3624
1801 / 3624
1901 / 3624
2001 / 3624
2101 / 3624
2201 / 3624
2301 / 3624
2401 / 3624
2501 / 3624
2601 / 3624
2701 / 3624
2801 / 3624
2901 / 3624
3001 / 3624
3101 / 3624
3201 / 3624
3301 / 3624
3401 / 3624
3501 / 3624
3601 / 3624
D:/Downloads/street-facilities-raw/images/12.sidewalk-block/good
1 / 1416
101 / 1416
201 / 1416
301 / 1416
401 / 1416
501 / 1416
601 / 1416

### .json 파일에서 라벨링 데이터 추출은 format_translation.ipynb 에서 수행